### 설치 – install



In [3]:
!pip install selenium

In [ ]:
pip install ipywidgets

     -------------------------------------- 123.4/123.4 kB 7.1 MB/s eta 0:00:00
     -------------------------------------- 245.3/245.3 kB 7.6 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 14.5 MB/s eta 0:00:00
  Using cached notebook-6.4.12-py3-none-any.whl (9.9 MB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
     ------------------------------------- 561.6/561.6 kB 11.7 MB/s eta 0:00:00
  Using cached Send2Trash-1.8.0-py3-none-any.whl (18 kB)
     ---------------------------------------- 73.3/73.3 kB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 59.5/59.5 kB 3.1 MB/s eta 0:00:00
     ------------------------------------- 160.9/160.9 kB 10.0 MB/s eta 0:00:00
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
Note: you may need to restart the kernel to use updated packages.Collecting defusedxml

     ---------------------------------------- 71.8/71.8 kB ? eta 0:00:00
     ---------------------------------------- 81.1/81.1 k

### 드라이버 – driver

In [4]:
!pip install chromedriver_autoinstaller

### 라이브러리 import

In [5]:
import pandas as pd
import numpy as np
import os
import math 

import requests
from bs4 import BeautifulSoup
import chromedriver_autoinstaller

# Selenium
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver import ActionChains as AC # ActionChains are a way to automate low level interactions such as mouse movements, mouse button actions, key press, and context menu interactions.
from selenium.webdriver.common.by import By       # Set of supported locator strategies.
from selenium.webdriver.common.keys import Keys   # Set of special keys codes
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait    # Constructor, takes a WebDriver instance and timeout in seconds.

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')


### 브라우저 열기 (chrome)

In [6]:
# 크롬창에서 멜론 발라드 부분 띄우기
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)

# 크롬 옵션
options = webdriver.ChromeOptions()  
# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=800, 1200") # window-size -> 기본 : 1920,1080

driver.get("https://www.melon.com/genre/song_list.htm?gnrCode=GN0100#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&params%5BsteadyYn%5D=N&po=pageObj&startIndex=1")
time.sleep(3)

### 🧙 전체 코드 (가사 없는 곡 PASS 가능)

In [9]:
songId = []
artist_names = []
songs = []
lyrics = []


### 페이지 이동 (1~1095 pages)
# p549 ~ p822 : 아연 range(548, 822)
for j in tqdm(range(549,550)) :  # PAGE 범위 조정할 것 (~1095 페이지까지 있음) 
    print(f'----------{j+1}page 크롤링 중----------')
    #차단 막기
    seed = np.random.randint(100)
    np.random.seed(seed)
    a = np.random.randint(5)
    time.sleep(a)
    
    # url 설정 (해당 페이지에 대한 url)
    page_url = f'https://www.melon.com/genre/song_list.htm?gnrCode=GN0100#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&params%5BsteadyYn%5D=N&po=pageObj&startIndex={1+(50*j)}'
    
    # url 접속
    driver.get(page_url)
    time.sleep(5)


    ### URL 이용하여 해당 페이지의 곡 번호 추출하기 (50 songs/page)
    # 가사 아이콘 확인하고 곡번호 url 확인하기
    elements = driver.find_elements(By.CSS_SELECTOR, "a.btn.button_icons.type03.song_info")
    
    product_id = []
    for ele in elements:
        thing = ele.get_attribute('href')
        p = re.compile('[0-9]{8,8}')
        song_id = p.findall(thing)[0]
        # print(song_id)
        product_id.append(song_id)

    print(product_id)
    songId.append(product_id)

    ### 곡 번호(songId)로 돌리기 in page
    for i in tqdm_notebook(product_id) :   # product_id
        print(f'----------{product_id.index(i)}번째 노래를 크롤링 중입니다.----------')
        #차단 막기
        seed = np.random.randint(100)
        np.random.seed(seed)
        a = np.random.randint(5)
        time.sleep(a)

        # url 설정 (해당 곡에 대한 url)
        url = f'https://www.melon.com/song/detail.htm?songId={i}'

        # url 접속
        driver.get(url)
        time.sleep(3)    

        # 가수명 크롤링
        artist_name = driver.find_element(By.CSS_SELECTOR, ".artist").text
        print(artist_name)
        artist_names.append(artist_name)
        
        # 곡명 크롤링
        song_name = driver.find_element(By.CSS_SELECTOR, "div.song_name").text
        print(song_name)
        songs.append(song_name)

            
        # 가사 있을 때
        try:
            # 펼치기 아이콘 클릭
            driver.find_element(By.CSS_SELECTOR, "button.button_more.arrow_d").click()
            time.sleep(3)

            # 가사 크롤링
            lyric = driver.find_element(By.ID, "d_video_summary").text
            print(lyric)
            lyrics.append(lyric)

        # 가사 없을 때
        except:
            lyric = "[가사 준비중]"
            print(lyric)
            lyrics.append(lyric)
        
            
# 가사 없는 곡 예시
# https://www.melon.com/song/detail.htm?songId=35413703  밤의 해변에서 혼자 (연주곡) / 초야
# https://www.melon.com/song/detail.htm?songId=35319615 꿈속으로 / Hope
# https://www.melon.com/song/detail.htm?songId=30811172 민들레 홀씨 되어 (cover.ver) / Various Artists


print('크롤링 완료!!!')

  0%|          | 0/1 [00:00<?, ?it/s]

----------550page 크롤링 중----------
['30810139', '30810513', '30810178', '30810136', '30809862', '30809861', '30809303', '30808547', '30808546', '30808341', '30808309', '30805733', '30805065', '30801582', '30810653', '30806728', '30810418', '30809766', '30804275', '30811172', '30809811', '30809801', '30809768', '30809595', '30809592', '30808989', '30808187', '30808029', '30807327', '30807306', '30807202', '30807121', '30807120', '30807105', '30806917', '30809165', '30809174', '30805546', '30807974', '30807819', '30809460', '30807666', '30806584', '30807949', '30807575', '30807018', '30799919', '30799719', '30809202', '30807936']


  0%|          | 0/50 [00:00<?, ?it/s]

----------0번째 노래를 크롤링 중입니다.----------
임세민
지나간다
다 지나간다 지나가
천천히 흘러 갈뿐야
눈물이 왈칵 쏟아질 것 같아도
다 지나간다 지나가
조금은 초라한 듯 보여도
네 한숨 같이 흩어질 거야 멀리
널 멈출 순 없지만
환하게 웃어줄게 난
막힌 가슴이 숨을 쉴 수 있도록
이 밤 끝엔
아침이 밝아오면
널 크게 불러줄게 난
따뜻한 햇살이 네게 비출 거야
다 지나간다 지나가
천천히 흘러 갈뿐야
어디에 닿을지는 아직 몰라도
다 지나간다 지나가
때로는 쓸쓸한 듯 보여도
맘 아파하지마 다 괜찮아 정말
널 멈출 순 없지만
환하게 웃어줄게 난
막힌 가슴이 숨을 쉴 수 있도록
이 밤 끝엔
아침이 밝아오면
널 크게 불러줄게 난
따뜻한 햇살이 네게 비출 거야
말뿐인 위로지만
가만히 안아줄게 난
지친 어깨가 잠시 쉴 수 있도록
이 밤 끝엔
아침이 밝아오면
네 손을 들어줄게 난
내일의 네 하루는 꼭 빛날 거야
----------1번째 노래를 크롤링 중입니다.----------
나슬
아프고 싶은 날
하루의 시작이
늘 불안해 잠들기 싫은 밤
생각하기 싫어서
온종일 아프고 싶은 날
내가 너무 미워
나와 한참 싸우고
어딘가에 있을
끝을 찾아 숨었죠
어느새 내 안은
너무 무거워져서
작은 몸부림도
허락할 수 없어요
천천히 고개를 들던
내 마음의 꽃이
혹시 시든 건 아닐까요
언젠가 꺾여진 꽃을
사랑스럽다고
믿어주면 덜 아플까요
날 향한 위로가
서럽게만 들리고
모진 말 들었던
기억 속에 갇혀요
어디쯤 왔을까
가늠조차 안돼서
제자리에 멈춰
한걸음도 못 가요
천천히 고개를 들던
내 마음의 꽃이
혹시 시든 건 아닐까요
언젠가 꺾여진 꽃을
사랑스럽다고
믿어주면 덜 아플까요
초라하기만 한 나의
작은 손으로도
눈물을 가릴 수 있나요
아프게 밟히던 내 맘
그 속 어딘가는
날 사랑하고 있을까요
----------2번째 노래를 크롤링 중입니다.----------
딜리버 (Deliver)
Welcome
어서 와 널 기다리고 있었어
아무 포장 없이
밤새 얘기하고 싶었어
진짜야 널 만나보

### 딕셔너리 & 데이터프레임 생성

In [10]:
page_coll = {}

page_coll['artist_name'] = artist_names
page_coll['song_name'] = songs
page_coll['lyric'] = lyrics

page_coll

{'artist_name': ['임세민',
  '나슬',
  '딜리버 (Deliver)',
  '이찬미',
  '김대현',
  '박진하',
  '정정란',
  '정유진',
  '미앤유 (me and you)',
  '세비야',
  '리마크',
  '오상아',
  '목영',
  '고은비',
  '윤종신, 정인',
  '이영훈',
  '아리밴드',
  '이시몬',
  '아날로그',
  'Various Artists',
  '정혜지',
  '이유나',
  '박연미',
  '브라우니 (Brownie)',
  'Kenji&ANi-',
  '미식가요',
  '톨게이트',
  '블루레이스 (Blu-Race)',
  '김우재',
  '사르나트 (Sarnath)',
  '마약강아지',
  '루오',
  '리시오',
  '네임 (Name)',
  '여디와 우림',
  '윤하 (YOUNHA)',
  '사무엘, 크리샤 츄 (Kriesha Chu)',
  '에릭남 (Eric Nam)',
  '폴킴',
  '김연지',
  '동해 (DONGHAE)',
  '박규리 (카라)',
  '한기란',
  '조인 (ZOIN)',
  '브라운 심리학과',
  '려',
  '홍찬미',
  '홍아',
  '한만재',
  '윤은기'],
 'song_name': ['지나간다',
  '아프고 싶은 날',
  'Welcome',
  '생각나다',
  '우리 이제',
  '매일 봐도 설레',
  '사랑했는데...',
  '숨결',
  '네가 없는 새벽 (With 박연 of 담소네공방)',
  '아파',
  '어차피',
  '그리다',
  '한순간',
  '서로 다른 이야기 속에..',
  '추위',
  '너에게',
  '널 기다리다',
  '내 오랜 꿈속의 그대에게',
  '겨울,그리고 우리 (With 김민경)',
  '민들레 홀씨 되어 (cover.ver)',
  '뭐해먹고 살지',
  '위로',
  '사랑하지 말아요',
  '토닥토닥',
  '헤어진 그날 눈이 내렸다',
  '문블렌딩',
  '쉬어가요',


In [11]:
target_df = pd.DataFrame.from_dict(page_coll, orient='index')
df = target_df.transpose()
df

,artist_name,song_name,lyric
0,임세민,지나간다,다 지나간다 지나가\n천천히 흘러 갈뿐야\n눈물이 왈칵 쏟아질 것 같아도\n다 지나...
1,나슬,아프고 싶은 날,하루의 시작이\n늘 불안해 잠들기 싫은 밤\n생각하기 싫어서\n온종일 아프고 싶은 ...
2,딜리버 (Deliver),Welcome,어서 와 널 기다리고 있었어\n아무 포장 없이\n밤새 얘기하고 싶었어\n진짜야 널 ...
3,이찬미,생각나다,하루 종일 생각나서\n한번 끄적여 봤어\n사랑한단 말 보고 싶단 말\n좋아한단 말\...
4,김대현,우리 이제,나 처음 그댈 본 날\n알 수 있었죠\n그대 내 사랑이 될 것을\n이제 내 눈을 봐...
5,박진하,매일 봐도 설레,제발 연락 좀 해줄래\n나는 온통 오빠 생각뿐\n답장이 없네 하루 종일\n기다리는 ...
6,정정란,사랑했는데...,술이라도 한잔 할까봐\n오늘은 널 잊도록\n누구라도 만나 볼까봐\n니 생각 못하도록...
7,정유진,숨결,왜 하필 그대인 거죠\n왜 지금 잊고 사는 내게 온 거죠\n힘들어요\n함께 할 수 ...
8,미앤유 (me and you),네가 없는 새벽 (With 박연 of 담소네공방),미안해 지금 자고 있는 중이니\n그냥 네 생각이 나서 연락해봤어\n보고 싶어\n눈을...
9,세비야,아파,아직 가시지 않은\n연기처럼\n뜨겁게 남아있는\n너란 이름의\n사랑 상처 자국\n여...


### .csv file 저장

In [12]:
# csv 파일로 저장
df.to_csv(f"melon_ballad({j+1}).csv", encoding='utf-8-sig')

In [13]:
# csv 파일로 저장 (범위)
# df.to_csv(f"melon_ballad({j+1}-{j+11}).csv", encoding='utf-8-sig')

In [14]:
pwd

'/Users/krc/Downloads'